<a href="https://colab.research.google.com/github/mario-rutman/preco-acoes-by-Claude-AI/blob/main/A%C3%A7%C3%B5es_by_Claude_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Lista de ações
acoes = ['ABEV3','B3SA3','BBAS3','BBDC3','BBSE3','BRAP4',
         'CMIG3','CPFE3','CPLE6','CXSE3','EGIE3','GRND3',
         'ITSA4','ITUB3','PETR4','PSSA3', 'SANB3', 'SANB4',
         'SAPR4', 'SHUL4','TAEE11','VIVT3','WEGE3','VALE3']

# Função para obter o nome completo da empresa
def obter_nome_empresa(ticker):
    try:
        stock = yf.Ticker(f"{ticker}.SA")
        return stock.info['longName']
    except:
        return "Nome não disponível"

# Data de início (5 anos atrás) e fim (hoje)
data_fim = datetime.now()
data_inicio = data_fim - timedelta(days=5*365)

# Lista para armazenar os dados
dados = []

# Obter dados para cada ação
for acao in acoes:
    ticker = f"{acao}.SA"
    stock = yf.Ticker(ticker)

    # Obter o histórico de preços dos últimos 5 anos
    hist = stock.history(start=data_inicio, end=data_fim)

    # Obter o nome completo da empresa
    nome_empresa = obter_nome_empresa(acao)

    # Adicionar dados para cada dia
    for data, row in hist.iterrows():
        dados.append({
            'Sigla': acao,
            'Nome': nome_empresa,
            'Data': data.strftime('%Y-%m-%d'),
            'Preço de Fechamento': round(row['Close'], 2)  # Arredondando para 2 casas decimais
        })

# Criar o DataFrame
df = pd.DataFrame(dados)

# Ordenar o DataFrame por Sigla e Data
df = df.sort_values(['Sigla', 'Data'])



In [ ]:
# Exibir as 10 primeiras e últimas linhas do DataFrame
print(df.head(10))
print(df.tail(10))

# Exibir informações sobre o DataFrame
print(df.info())

# Salvar o DataFrame em um arquivo CSV (opcional)
# df.to_csv('acoes_brasileiras_5anos.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar o estilo do seaborn para melhor aparência
sns.set_style("whitegrid")

# Calcular o número de linhas e colunas para o subplot
n_siglas = len(df['Sigla'].unique())
n_cols = 4  # Você pode ajustar isso conforme necessário
n_rows = -(-n_siglas // n_cols)  # Arredonda para cima

# Criar a figura e os subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
# fig.suptitle('Boxplots de Preços de Fechamento por Ação', fontsize=16)

# Achatar o array de axes para facilitar a iteração
axes = axes.flatten()

# Iterar sobre cada sigla
for i, sigla in enumerate(sorted(df['Sigla'].unique())):
    ax = axes[i]
    data = df[df['Sigla'] == sigla]

    # Obter o nome da empresa
    nome = data['Nome'].iloc[0]

    # Criar o boxplot e configurar estilo
    sns.boxplot(x='Sigla', y='Preço de Fechamento', data=data, ax=ax,
            medianprops={"color": "green", "linewidth": 3},
            boxprops={"facecolor": '#f2f2f2', "edgecolor": "black", "linewidth": 1.5},
            whiskerprops={"color": "black"},
            capprops={"color": "black"},
            flierprops={"markeredgecolor": "black"})

    # Adicionar a linha do último preço de fechamento
    ultimo_preco = data['Preço de Fechamento'].iloc[-1]
    ax.axhline(y=ultimo_preco, color='red', linestyle='--', linewidth=3)


    # Adicionar anotação do último preço
    ax.annotate(f'{ultimo_preco:.2f}',
                xy=(0, ultimo_preco),
                xytext=(2, 2),
                textcoords='offset points',
                ha='left', va='bottom',
                fontsize=12,
                weight='bold')


    # Configurar o título e rótulos
    ax.set_title(f'{sigla} - {nome}', fontsize=12, wrap=True)
    ax.set_xlabel('')
    ax.set_ylabel('Preço de Fechamento' if i % n_cols == 0 else '')

    # Remover os ticks do eixo x
    ax.set_xticks([])

# Remover subplots extras
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Ajustar o layout
plt.tight_layout()

# Mostrar o gráfico
plt.show()

# Opcional: Salvar o gráfico
# plt.savefig('boxplots_acoes.png', dpi=300, bbox_inches='tight')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calcular o número de linhas e colunas para o subplot
n_siglas = len(df['Sigla'].unique())
n_cols = 6  # Você pode ajustar isso conforme necessário
n_rows = -(-n_siglas // n_cols)  # Arredonda para cima

# Criar a figura com subplots
fig = make_subplots(rows=n_rows, cols=n_cols,
                    subplot_titles=[f"{sigla}" # - {df[df['Sigla'] == sigla]['Nome'].iloc[0]}"
                                    for sigla in sorted(df['Sigla'].unique())],
                    vertical_spacing=0.05,
                    horizontal_spacing=0.05)

# Iterar sobre cada sigla
for i, sigla in enumerate(sorted(df['Sigla'].unique())):
    row = i // n_cols + 1
    col = i % n_cols + 1

    data = df[df['Sigla'] == sigla]

    # Criar o boxplot
    fig.add_trace(go.Box(
        y=data['Preço de Fechamento'],
        name=sigla,
        boxmean=False,  # Retira linha média.
        fillcolor='#f2f2f2',
        line=dict(color='black'),
        marker=dict(color='black'),
        whiskerwidth=1,  # Change whiskerwidth to be between 0 and 1
        boxpoints='outliers',
        hoverinfo='skip',  # Mostra apenas o valor y no hover
        hoverlabel=dict(namelength=0)  # Remove o nome da variável do hover
    ), row=row, col=col)




    # Adicionar a linha do último preço de fechamento
    ultimo_preco = data['Preço de Fechamento'].iloc[-1]
    fig.add_shape(
        type="line",
        x0=-0.5, x1=0.5,
        y0=ultimo_preco, y1=ultimo_preco,
        line=dict(color="red", width=3, dash="dash"),
        row=row, col=col
    )

     # Adicionar anotação com o valor do último preço de fechamento
    fig.add_annotation(
        x=-0.5,
        y=ultimo_preco,
        text=f"{ultimo_preco:.2f}",
        showarrow=False,
        font=dict(size=14, color="black", family="Arial Black"),
        xanchor="left",
        yanchor="bottom",
        #bgcolor="yellow",
        #opacity=0.8,
        row=row,
        col=col
    )

# Atualizar o layout
fig.update_layout(
    height=350 * n_rows,
    width=300 * n_cols,
    showlegend=False,
    title_text="",
    # title_font=dict(size=16),
)

# Atualizar os eixos
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(title_text="Preço de Fechamento", row=1, col=1)

# Mostrar o gráfico
fig.show()

# Opcional: Salvar o gráfico
# fig.write_image("boxplots_acoes.png", scale=2)

# Nova seção